<h1>
<center>
Module 8: K-NN
</center>
</h1>
<div class=h1_cell>
<p>
I kind of like K Nearest Neighbor (K-NN). It is a version of crowd-sourcing like random forests. The general idea is that you choose K to be how many "consultants" you take votes from. With random forests, the consultants were decision trees. In K-NN, they are the other rows in the table. So let's say you are trying to predict what value to give for a row i. What you do is check all the other rows (excluding i) to get their distance from i. You choose the K closest. You take the predictions from each of those K. Majority wins and is the prediction you use for row i.
<p>
Notice anything interesting? We are not doing any training! We do have to have labeled data to get the K consultants. But no training step involved. Wow.
<p>
However, there is always a catch. Think about what we are doing. For every row, we are calculating distance to every other row. Feels `O(n)` to me. And remember, the shelter data we looked at had 26K rows! What is cost of decision tree or random forest to make a prediction? Worst case it is the depth of the tree, i.e., it is constant speed. If we need to make real-time predictions then `O(n)` might not be good enough.
</div>
<p>
  And btw, when testing K-NN, our cost will balloon to `O(n**2)`.

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/class_tables/loan_table_week4.csv', 'r') as f:
  loan_table = pd.read_csv(f)

In [0]:
!rm library_w19_week6.py

In [4]:
from google.colab import files
files.upload()

Saving library_w19_week6.py to library_w19_week6.py


{'library_w19_week6.py': b'import pandas as pd\r\nimport numpy as np\r\nfrom functools import reduce\r\nfrom types import SimpleNamespace\r\nimport random\r\n\r\ndef predictor_case(row, pred, target):\r\n  case_dict = {(0,0): \'true_negative\', (1,1): \'true_positive\', (0,1): \'false_negative\', (1,0): \'false_positive\'}\r\n  actual = row[target]\r\n  prediction = row[pred]\r\n  case = case_dict[(prediction, actual)]\r\n  return case\r\n\r\ndef informedness(cases):\r\n  tp = 0\r\n  if \'true_positive\' in cases:\r\n    tp = cases[\'true_positive\']\r\n  tn = 0\r\n  if \'true_negative\' in cases:\r\n    tn = cases[\'true_negative\']\r\n  fp = 0\r\n  if \'false_positive\' in cases:\r\n    fp = cases[\'false_positive\']\r\n  fn = 0\r\n  if \'false_negative\' in cases:\r\n    fn = cases[\'false_negative\']\r\n  if (((tp+fn) == 0) or ((tn+fp) == 0)):\r\n    return -1\r\n  else:\r\n    recall = 1.0*tp/(tp+fn)\r\n    specificity = 1.0*tn/(tn+fp)\r\n    J = (recall + specificity) - 1\r\n    

In [5]:
from library_w19_week6 import *

%who function

accuracy	 build_pred	 build_tree_iter	 compute_prediction	 compute_training	 f1	 find_best_splitter	 forest_builder	 forest_scores	 
generate_table	 gig	 gini	 informedness	 k_fold	 k_fold_random	 path_id	 predictor_case	 probabilities	 
produce_scores	 reorder_paths	 shuffle	 tree_predictor	 verify_unique	 vote_taker	 


In [6]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low,1,0,0,0,0,1,0,0,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0,1,0,0,0,0


In [7]:
loan_table.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_low,apin_average,apin_high,apin_nan,dep_0,dep_1,dep_2,dep_3+,dep_nan
count,614.000000,614.000000,592.000000,600.00000,564.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.0,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.0,614.000000,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199,0.687296,0.035831,146.412162,0.291531,0.379479,0.328990,0.0,0.905537,0.074919,0.019544,0.144951,0.773616,0.081433,0.988599,0.008143,0.003257,0.0,0.561889,0.166124,0.164495,0.083062,0.024430
std,6109.041673,2926.248369,85.587325,65.12041,0.364878,0.463973,0.186019,84.037468,0.454838,0.485653,0.470229,0.0,0.292710,0.263475,0.138540,0.352339,0.418832,0.273722,0.106250,0.089945,0.057026,0.0,0.496559,0.372495,0.371027,0.276201,0.154506
min,150.000000,0.000000,9.000000,12.00000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000,0.000000,0.000000,100.250000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000,1.000000,0.000000,129.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000,1.000000,0.000000,164.750000,1.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000,1.000000,1.000000,700.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000


<h1>You are mostly on your own</h1>

I am going to give you the freedom to implement the K-NN components. My only constraints are that you implement these functions:
<pre>
def euclidean_distance(vector1, vector2):  #euclidean distance between 2 vectors

def knn(row_index, table, k, columns, target):  #provide prediction for row given K-NN algorithm. Only use values in columns.

def knn_tester(table, k, columns, target): #produce predictions for every row in table according to columns. Return accuracy score. Danger: O(n**2).
</pre>


I am calling these "splitter" columns but that is a misnomer. Splitter has to do with decision trees. Should rename them at some point.

In [0]:
splitter_columns = [
        #Dependents
        'dep_0', 'dep_1', 'dep_2', 'dep_3+',
        #ApplicantIncome
       'ApplicantIncome', 'CoapplicantIncome',
        #Property_Area
        'pa_Rural', 'pa_Semiurban','pa_Urban',
        #LoanAmount
        'filled_lam',
        #Credit_History
        'ch_bad', 'ch_good']

In [0]:
target = 'Loan_Status'

In [0]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd
import operator

#row_index is the row we want to test
#table is 'loan_table'
#k is number of neighbors we take
#columns is 'splitter_columns'
#target is 'Loan_Status'
def euclidean_distance(vector1, vector2):
  return np.sqrt(sum([(a - b) ** 2 for a, b in zip(vector1, vector2)]))

def knn(row_index, table, columns, k, target):
  dist = []
  init_row = table[columns].values[row_index].tolist() #values instead of iloc
  for index, row in table.iterrows():
    if index == row_index:
      continue
    dist.append((index, euclidean_distance(init_row, row[columns].tolist())))
  dist.sort(key=operator.itemgetter(1))
  vote = []
  for (x, y) in dist[:k]:
    vote.append(table[target].values[x]) #same here
  return max(set(vote), key=vote.count)

def knn_tester(table, k, columns, target):
  all_votes = []
  for i in range(len(table)):
    pred = knn(i, table, columns, k, target)
    all_votes.append(pred)
  table['all_votes'] = all_votes
  table['vote_type'] = table.apply(lambda row: predictor_case(row, pred='all_votes', target=target), axis=1)
  p1_types = table['vote_type'].value_counts()
  return accuracy(p1_types)

<h2>Match my results</h2>

Start with single predictions.

In [15]:
#test with row 1, k=5
import time

start = time.time()
    
prediction_row0 = knn(1, loan_table,splitter_columns,5, target)        
print('prediction: ' + str(prediction_row0))  #actual is 0
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

prediction: 1
elapsed time: 0.38669562339782715


In [16]:
#test with row 1, k=11

import time

start = time.time()
    
prediction_row0 = knn(1, loan_table,splitter_columns, 11, target)        
print('prediction: ' + str(prediction_row0))
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

prediction: 1
elapsed time: 0.39823150634765625


<h2>Now to testing</h2>

knn_tester should use knn to make predictions for every row. Check predictions against target and compute and return accuracy.
<p>
All of my functions are nothing fancy. And it costs me about 2 minutes per run.

In [17]:
#k=5 (about 2 minutes on colab)

start = time.time()
    
the_accuracy = knn_tester(loan_table, 5, splitter_columns, target)        
print('accuracy: ' + str(the_accuracy))
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

accuracy: 0.6302931596091205
elapsed time: 230.43174386024475


<h2>Not that great</h2>

Why so bad? Normally K-NN fails when there are too many features/columns in play. It relies on distances, and the more columns you add, the harder to discern distance differences. This is an example of the curse of dimensionality (jargon alert). It is kind of counter-intuitive: the more information (features/columns) you add, the worse you do. Here is an intro to the problem: http://www.visiondummy.com/2014/04/curse-dimensionality-affect-classification/. Let's reduce our columns and see if it makes any difference.


In [0]:
fewer_columns = [
        #ApplicantIncome
       'ApplicantIncome', 'CoapplicantIncome',
        #LoanAmount
        'filled_lam',
        #Credit_History
        'ch_bad', 'ch_good']

In [19]:
#k=5 (about 2 minutes on colab)

start = time.time()
    
the_accuracy = knn_tester(loan_table, 5, fewer_columns, target)        
print('accuracy: ' + str(the_accuracy))
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

accuracy: 0.6302931596091205
elapsed time: 227.48288750648499


Exactly the same. Let's try some different values for k.

In [20]:
#k = 11, full columns
start = time.time()
    
the_accuracy = knn_tester(loan_table, 11, splitter_columns, target)        
print('accuracy: ' + str(the_accuracy))
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

accuracy: 0.6628664495114006
elapsed time: 232.124009847641


Got a bit of improvement. One more.

In [21]:
#k = 3, full columns
start = time.time()
    
the_accuracy = knn_tester(loan_table, 3, splitter_columns, target)        
print('accuracy: ' + str(the_accuracy))
end = time.time()
print('elapsed time: '+ str(end - start))  # in seconds

accuracy: 0.6042345276872965
elapsed time: 233.64303064346313


Heading back down in accuracy. I'll stop here but will leave you with an extra credit assignment (optional).

<h2>Update your library</h2>

Name it `library_w19_week7.py`.

<h2>Extra Credit</h2>

Beat my testing times. You can use any of the pandas and numpy methods. As reminder, here were my times (approximately) using brute force approach:
<pre>
knn_tester(loan_table, 5, splitter_columns, target) : 107ish
knn_tester(loan_table, 5, fewer_columns, target) :  83ish
knn_tester(loan_table, 11, splitter_columns, target) : 108ish
knn_tester(loan_table, 3, splitter_columns, target) : 108ish
</pre>
<p>
  Here is a time I got by a fairly simple trick:
  <pre>
  knn_tester(loan_table, 3, splitter_columns, target) 
  accuracy: 0.6042345276872965
elapsed time: 4.287370681762695
</pre>
So from 108 to 4. Pretty dang good.